In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)

df_total_coste = pd.read_parquet(f'../data/raw/df_total_coste.parquet')
df_total_indicadores = pd.read_parquet(f'../data/raw/df_total_indicadores.parquet')


df_coste = pd.read_parquet(f'../data/processed/df_coste.parquet')
df_indicadores = pd.read_parquet(f'../data/processed/df_indicadores.parquet')

In [15]:
df_coste.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336690 entries, 0 to 17939
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   Provincia              336690 non-null  category
 1   Código Ente Principal  336690 non-null  category
 2   Tipo Ente Principal    336690 non-null  category
 3   Nombre Ente Principal  336690 non-null  category
 4   Descripción            336690 non-null  category
 5   coste_efectivo         336690 non-null  float64 
dtypes: category(5), float64(1)
memory usage: 8.5 MB


In [16]:
df_coste.nunique()

Provincia                   46
Código Ente Principal     5570
Tipo Ente Principal          7
Nombre Ente Principal     5554
Descripción                 43
coste_efectivo           70818
dtype: int64

In [17]:
df_indicadores.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 582717 entries, 0 to 27677
Data columns (total 7 columns):
 #   Column                          Non-Null Count   Dtype   
---  ------                          --------------   -----   
 0   Provincia                       582717 non-null  category
 1   Código Ente Principal           582717 non-null  category
 2   Tipo Ente Principal             582717 non-null  category
 3   Nombre Ente Principal           582717 non-null  category
 4   Descripción                     582717 non-null  category
 5   Unidades físicas de referencia  582717 non-null  category
 6   Nº unidades                     582717 non-null  float64 
dtypes: category(6), float64(1)
memory usage: 14.5 MB


In [18]:
df_indicadores.nunique()

Provincia                            46
Código Ente Principal              5570
Tipo Ente Principal                   7
Nombre Ente Principal              5554
Descripción                          43
Unidades físicas de referencia       59
Nº unidades                       35382
dtype: int64

In [19]:
df_coste.columns

Index(['Provincia', 'Código Ente Principal', 'Tipo Ente Principal',
       'Nombre Ente Principal', 'Descripción', 'coste_efectivo'],
      dtype='object')

In [20]:
peso=df_coste.groupby(by='Tipo Ente Principal').sum()

In [21]:
df_coste['coste_efectivo'].sum()

461049090058.33984

In [22]:
peso.apply(lambda x: round(((x/df_coste['coste_efectivo'].sum())*100),2))

,coste_efectivo
Tipo Ente Principal,
Agrupación de municipios,0.00
Ayuntamiento,98.85
Comarca,0.05
Diputación/Consejo/Cabildo,0.64
Entidad Metropolitana,0.36
Entidad de ámbito territorial inferior al municipio,0.01
Mancomunidad,0.09


In [23]:
df_coste.loc[(df_coste['Descripción']=='Abastos, mercados, lonjas ')&(df_coste['Nombre Ente Principal']=='Huércal de Almería'),'coste_efectivo']=0

In [24]:
df_coste.loc[(df_coste['Descripción']=='Abastos, mercados, lonjas ')&(df_coste['Nombre Ente Principal']=='Huércal de Almería'),'coste_efectivo']

1528    0.0
Name: coste_efectivo, dtype: float64

In [46]:
df_coste['coste_efectivo'].sum()

37601533795.200005

In [26]:
peso=df_coste.groupby(by='Tipo Ente Principal').sum()

In [27]:
peso.apply(lambda x: round(((x/df_coste['coste_efectivo'].sum())*100),2))

,coste_efectivo
Tipo Ente Principal,
Agrupación de municipios,0.00
Ayuntamiento,85.92
Comarca,0.63
Diputación/Consejo/Cabildo,7.87
Entidad Metropolitana,4.36
Entidad de ámbito territorial inferior al municipio,0.07
Mancomunidad,1.15


In [28]:
# df_coste['cod_ente']=df_coste['Código Ente Principal']
# df_coste['cod_ente'] = df_coste['cod_ente'].str.replace(r'\d{2}-\d{2}-\d{3}-', '').str.replace(r'-\d{3}$', '')


In [29]:
# columns=['Provincia', 'Código Ente Principal', 'Tipo Ente Principal',
#        'Nombre Ente Principal', 'Descripción']

df_coste_pivot=df_coste.pivot_table(index=['Código Ente Principal'],values=['coste_efectivo'],columns=['Descripción'],aggfunc=np.sum)
df_coste_pivot.columns = df_coste_pivot.columns.droplevel()
df_coste_pivot.columns = df_coste_pivot.columns.tolist()
df_coste_pivot.columns
df_coste_pivot.reset_index(inplace=True)



In [30]:
more_columns=df_coste.groupby(by=['Código Ente Principal',])[['Provincia','Tipo Ente Principal','Nombre Ente Principal']].min().reset_index()

In [31]:
more_columns.nunique()

Código Ente Principal    5570
Provincia                  46
Tipo Ente Principal         7
Nombre Ente Principal    5554
dtype: int64

In [32]:
df_coste_pivot['Provincia']=more_columns['Provincia']
df_coste_pivot['Tipo Ente Principal']=more_columns['Tipo Ente Principal']
df_coste_pivot['Nombre Ente Principal']=more_columns['Nombre Ente Principal']






In [33]:
cols = df_coste_pivot.columns.tolist()
cols = cols[-3:] + cols[:-3] 


df_coste_pivot= df_coste_pivot[cols]

In [34]:
pd.set_option('display.max_columns', 10)

df_coste_pivot['codigoM']=df_coste_pivot['Código Ente Principal']

df_coste_pivot['codigoM'] = df_coste_pivot['codigoM'].str.replace(r'^.{3}|.{7}$|-', '')










In [35]:
df_coste_pivot['Tipo Ente Principal'].value_counts()

Ayuntamiento                                           4869
Mancomunidad                                            311
Entidad de ámbito territorial inferior al municipio     280
Comarca                                                  53
Diputación/Consejo/Cabildo                               47
Agrupación de municipios                                  7
Entidad Metropolitana                                     3
Name: Tipo Ente Principal, dtype: int64

In [36]:
df_final=df_coste_pivot.loc[(df_coste_pivot['Tipo Ente Principal']=='Ayuntamiento')|\
                      (df_coste_pivot['Tipo Ente Principal']=='Entidad de ámbito territorial inferior al municipio')].reset_index()




df_final=df_final.drop(columns=['index'])

df_final




,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,Abastecimiento domiciliario de agua potable,...,Transporte colectivo urbano de viajeros,Tratamiento de residuos,"Tráfico, estacionamiento de vehículos y movilidad","Urbanismo: planeamiento, gestión, ejecución y disciplina urbanística",codigoM
0,Almería,Ayuntamiento,Abla,01-04-001-AA-000,282928.39,...,0.0,0.0,4260.16,2275.98,04001
1,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,77227.64,...,0.0,0.0,0.00,16674.59,04002
2,Almería,Ayuntamiento,Adra,01-04-003-AA-000,1255574.16,...,0.0,0.0,144618.30,125720.58,04003
3,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,60828.11,...,0.0,0.0,0.00,5637.62,04004
4,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,157266.48,...,0.0,0.0,0.00,0.00,04005
5,Almería,Ayuntamiento,Albox,01-04-006-AA-000,0.00,...,0.0,0.0,75471.49,259966.22,04006
6,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,61945.05,...,0.0,0.0,0.00,18720.14,04007
7,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,50007.27,...,0.0,0.0,0.00,41055.51,04008
8,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,36579.39,...,0.0,0.0,0.00,0.00,04009
...,...,...,...,...,...,...,...,...,...,...,...


## POBLACION

In [37]:
df_pob=pd.read_excel(f'/home/carpiero/IRONHACK/Proyecto FINAL/Datos municipios.xlsx', sheet_name=0,header=0)



In [38]:
df_pob=df_pob[['Población 2018',  'superficie',  'entidades singulares menores', ' ']]

In [39]:
df_pob['codigoM']=df_pob[' ']

In [40]:
df_pob['codigoM'] = df_pob['codigoM'].str.replace(r'\D', '')

df_pob

,Población 2018,superficie,entidades singulares menores,,codigoM
0,2887,19.95,2.0,01001 Alegría-Dulantzi,01001
1,10286,96.36,10.0,01002 Amurrio,01002
2,1483,73.27,9.0,01003 Aramaio,01003
3,1831,27.45,7.0,01004 Artziniega,01004
4,219,12.94,3.0,01006 Armiñón,01006
5,982,57.41,11.0,01008 Arratzua-Ubarrundia,01008
6,1643,65.18,10.0,01009 Asparrena,01009
7,2967,140.85,24.0,01010 Ayala/Aiara,01010
8,286,9.46,1.0,01011 Baños de Ebro/Mañueta,01011
...,...,...,...,...,...


In [41]:
df_final_pob = pd.merge(df_final, df_pob, on='codigoM', how='left')

In [42]:
df_final_pob=df_final_pob.drop(columns=['superficie',
       'entidades singulares menores', ' '])





In [79]:
df_final_pob['TOTAL']=range(df_final_pob.shape[0])


df_final_pob['TOTAL']=df_final_pob.apply(lambda new: df_final_pob.iloc[new['TOTAL'],4:47].sum(), axis=1)






In [81]:
df_final_pob['TOTAL'].sum()

32335651226.0

In [84]:
df_final_pob[['Tipo Ente Principal','TOTAL']].groupby('Tipo Ente Principal').sum()

,TOTAL
Tipo Ente Principal,
Ayuntamiento,3.230883e+10
Entidad de ámbito territorial inferior al municipio,2.682528e+07


In [50]:
# fig=px.histogram(df_final_pob, x='Población 2018',nbins=400,template='seaborn',title='Age',barmode='group')
# fig.show()

In [53]:

df_final_pob.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5149 entries, 0 to 5148
Data columns (total 49 columns):
 #   Column                                                                                                                                                                             Non-Null Count  Dtype   
---  ------                                                                                                                                                                             --------------  -----   
 0   Provincia                                                                                                                                                                          5149 non-null   object  
 1   Tipo Ente Principal                                                                                                                                                                5149 non-null   object  
 2   Nombre Ente Principal                               

In [33]:
lo=df_coste.groupby(by=['Descripción','Nombre Ente Principal'],as_index=False).sum()

lo2=lo.loc[(lo['Descripción']=='Abastos, mercados, lonjas ')].sort_values(by='coste_efectivo',ascending=False)

In [34]:
lo2.to_excel(f'/home/carpiero/IRONHACK/zOld/lo.xlsx', index=False)

In [ ]:
df_indicadores.loc[(df_indicadores['Descripción']=='Actividades funerarias')&(df_indicadores['Código Ente Principal']=='17-46-904-AA-000')]